In [1]:
import sys
sys.path.insert(0, '../code/')
from catalog import get_tois
df = get_tois()

Saved:  ../data/TOIs.csv


In [2]:
df.shape

(6160, 62)

In [84]:
import pandas as pd

data = pd.read_csv('../data/youth_indicators_tois.csv')
data

,TOI,bv,bv_err,li,prot,rhk,teff
0,101.01,0.656000,0.1,NaN,1.430321,NaN,5648.642857
1,102.01,0.507667,0.1,NaN,4.410011,NaN,6257.270417
2,103.01,0.291000,0.1,NaN,3.554638,NaN,6086.159167
3,104.01,0.545667,0.1,NaN,4.090360,NaN,6046.516250
4,105.01,0.687667,0.1,NaN,2.187803,NaN,5741.285333
...,...,...,...,...,...,...,...
4312,5023.01,NaN,NaN,NaN,NaN,NaN,3720.494840
4313,5024.01,0.612000,NaN,NaN,NaN,NaN,5985.393333
4314,5025.01,0.313000,NaN,NaN,NaN,NaN,8132.166667
4315,5026.01,0.700000,NaN,NaN,NaN,NaN,6444.962144


In [89]:
data.dropna(how='any')

,TOI,bv,bv_err,li,prot,rhk,teff
98,200.01,0.735647,0.048146,108.107498,5.475,-4.09,5482.444615


In [88]:
for c in data.columns:
    print(c,len(data[c].dropna()))

TOI 4429
bv 4102
bv_err 4000
li 10
prot 595
rhk 17
teff 4421


In [ ]:
import pandas as pd
import numpy as np
from astropy.coordinates import SkyCoord
from tqdm.notebook import tqdm
from catalog import Target

params = ["B-V", "log_R_HK_", "EWLi", "Teff", "Prot"]

for i,row in tqdm(df.iterrows()):
    if data.TOI.isin([row.TOI]).sum()==0:
        d = {}
        d['TOI'] = row.TOI

        coords = SkyCoord(row.RA, row.Dec, unit=('hourangle','deg'))
        t = Target(coords.ra.deg, coords.dec.deg)

        for param in params:
            p = t.query_vizier_param(param)
            v = np.nanmean(list(p.values()))
            d[param] = v
        s = pd.Series(d).rename({"B-V": "bv", 
                 "log_R_HK_": "rhk", 
                 "EWLi": "li", 
                 "Teff": "teff", 
                 "Prot": "prot"}).to_frame().T
        data = pd.concat([data,s], ignore_index=True) 

In [ ]:
data.to_csv('../data/youth_indicators_tois.csv', index=False)